In [ ]:
import mlflow

from anaconda.enterprise.server.common.sdk import load_ae5_user_secrets
from workflow.utils.tracking import upsert_experiment

# Ensure:
#  1. We load AE5 secrets
#  2. That we have set our experiment name for reporting.
#     See notes in anaconda-project.xml around MLFlow project naming control.

load_ae5_user_secrets()
mlflow.set_experiment(experiment_id=upsert_experiment())

In [ ]:
# Define our parameters:

# The prompt to use for image generation.
prompt: str = ""

# The base data directory that requests are stored in.
data_base_dir: str = "data"

# Number of total images to generate.
total_batch_size: int = 1

# Number of images to generate per worker invocation.
per_worker_batch_size: int = 1

# Image Width
image_width: int = 512

# Image Height
image_height: int = 512

# The name of the run.
run_name: str = "workflow-step-process-data"

# Flag for appending a unique string to the end of run names.
unique: bool = True

# The backend to use for workers.
backend: str = 'local'

In [ ]:
from mlflow import ActiveRun
from workflow.utils.tracking import build_run_name

workflow_run: ActiveRun = mlflow.start_run(run_name=build_run_name(name=run_name, unique=unique))

In [ ]:
import uuid
from pathlib import Path

#############################################################################
# Set up runtime environment
#############################################################################

print(f"prompt={prompt}")
print(f"data_base_dir={data_base_dir}")
print(f"total_batch_size={total_batch_size}")
print(f"per_worker_batch_size={per_worker_batch_size}")
print(f"image_width={image_width}")
print(f"image_height={image_height}")
print(f"backend={backend}")

run_id: str = workflow_run.info.run_id
print(f"run_id: {run_id}")

request_id: str = str(uuid.uuid4())
base_path: Path = Path(data_base_dir) / request_id
base_path.mkdir(parents=True, exist_ok=True)
with open(file=(base_path / "prompt.txt").as_posix(), mode="w", encoding="utf-8") as file:
    file.write(prompt)

In [ ]:
from mlflow_adsp import ADSPScheduler, ExecuteStepRequest

#############################################################################
# Prepare Worker Environment Step
#############################################################################
ADSPScheduler.execute_step(
    request=ExecuteStepRequest(
        entry_point="prepare_worker_environment",
        parameters={"backend": backend},
        run_name=build_run_name(name="workflow-step-prepare-worker-environment", unique=unique),
        synchronous=True,
        backend="local",
    )
)

In [ ]:
from typing import List
import math

#############################################################################
# Processing Step [Build Requests]
#############################################################################
worker_count: int = math.ceil(total_batch_size / per_worker_batch_size)
print(f"number of workers: {worker_count}")

jobs: List[ExecuteStepRequest] = []
for _ in range(worker_count):
    request: ExecuteStepRequest = ExecuteStepRequest(
        entry_point="process_data",
        parameters={
            "request_id": request_id,
            "data_base_dir": data_base_dir,
            "batch_size": per_worker_batch_size,
            "image_width": image_width,
            "image_height": image_height,
        },
        run_name=build_run_name(name="workflow-step-process-data", unique=unique),
        backend=backend,
        backend_config={"resource_profile": "large"},
        synchronous=backend == "local",  # Force to serial processing if running locally.
    )
    jobs.append(request)

In [ ]:
from mlflow_adsp import ADSPMetaJob

# submit jobs
print("starting workers")
adsp_jobs: List[ADSPMetaJob] = ADSPScheduler().process_work_queue(requests=jobs)
print("Step execution completed")

In [ ]:
# Review job status
for job in adsp_jobs:
    print(f"Job ID: {job.id}, Status: {job.last_seen_status}, Number of executions: {len(job.runs)}")